In [1]:
import pandas as pd
import statsmodels.api as sm

In [3]:
sales_data = pd.read_csv("/Users/miloszubac/Downloads/Chocolate Sales 2.csv") 
chocolate_nutrition = pd.read_csv("/Users/miloszubac/Downloads/Chocolate Nutrition Facts.csv") 

In [5]:
sales_data.head()

,Sales Person,Country,Product,Date,Amount,Boxes Shipped
0,Jehu Rudeforth,UK,Mint Chip Choco,04-Jan-22,5320,180
1,Van Tuxwell,India,85% Dark Bars,01-Aug-22,7896,94
2,Gigi Bohling,India,Peanut Butter Cubes,07-Jul-22,4501,91
3,Jan Morforth,Australia,Peanut Butter Cubes,27-Apr-22,12726,342
4,Jehu Rudeforth,UK,Peanut Butter Cubes,24-Feb-22,13685,184


In [6]:
# Load data
df = pd.merge(sales_data, chocolate_nutrition, left_on='Product', right_on='product_name', how='inner')
df.head()

,Sales Person,Country,Product,Date,Amount,Boxes Shipped,product_name,serving_size_g,calories,total_fat_g,saturated_fat_g,carbohydrates_g,sugars_g,protein_g,fiber_g
0,Jehu Rudeforth,UK,Mint Chip Choco,04-Jan-22,5320,180,Mint Chip Choco,30.0,210,10.0,5.0,15.0,10.0,2.0,3.0
1,Brien Boise,New Zealand,Mint Chip Choco,01-Aug-22,63,181,Mint Chip Choco,30.0,210,10.0,5.0,15.0,10.0,2.0,3.0
2,Camilla Castle,New Zealand,Mint Chip Choco,13-Jun-22,1162,190,Mint Chip Choco,30.0,210,10.0,5.0,15.0,10.0,2.0,3.0
3,Ches Bonnell,Australia,Mint Chip Choco,21-Feb-22,9660,92,Mint Chip Choco,30.0,210,10.0,5.0,15.0,10.0,2.0,3.0
4,Dennison Crosswaite,New Zealand,Mint Chip Choco,05-Apr-22,6454,417,Mint Chip Choco,30.0,210,10.0,5.0,15.0,10.0,2.0,3.0


In [7]:
df['calories_per100g'] = df['calories']/df['serving_size_g'] * 100
df['sugar_per100g'] = df['sugars_g']/df['serving_size_g'] * 100

In [9]:
canada_df = df[df['Country'] == 'Canada']

In [16]:
canada_df_agg = canada_df.groupby('Product', as_index=False).agg({'Amount': 'sum', 'calories_per100g': 'first', 'sugar_per100g': 'first'})
canada_df_agg

,Product,Amount,calories_per100g,sugar_per100g
0,50% Dark Bites,45115,560.000000,32.000000
1,70% Dark Bites,59024,400.000000,24.000000
2,85% Dark Bars,37926,600.000000,16.666667
3,99% Dark & Pure,44198,600.000000,10.000000
4,After Nines,41993,566.666667,40.000000
5,Almond Choco,42028,550.000000,30.000000
6,Baker's Choco Chips,36456,535.714286,28.571429
7,Caramel Stuffed Bars,33376,622.222222,48.888889
8,Choco Coated Almonds,57463,366.666667,33.333333
9,Drinking Coco,47964,450.000000,50.000000


In [17]:
X = canada_df_agg[['calories_per100g']]  
X = sm.add_constant(X)  
y = canada_df_agg['Amount']  

# Fit regression model
model = sm.OLS(y, X).fit()

# Print results
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                 Amount   R-squared:                       0.423
Model:                            OLS   Adj. R-squared:                  0.394
Method:                 Least Squares   F-statistic:                     14.64
Date:                Tue, 18 Mar 2025   Prob (F-statistic):            0.00106
Time:                        11:18:14   Log-Likelihood:                -234.68
No. Observations:                  22   AIC:                             473.4
Df Residuals:                      20   BIC:                             475.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const             8.787e+04   1.18e+04  

In [20]:
X = canada_df_agg[['sugar_per100g']]  
X = sm.add_constant(X)  
y = canada_df_agg['Amount']  

# Fit regression model
model = sm.OLS(y, X).fit()

# Print results
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 Amount   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                 -0.029
Method:                 Least Squares   F-statistic:                    0.4051
Date:                Tue, 18 Mar 2025   Prob (F-statistic):              0.532
Time:                        11:22:00   Log-Likelihood:                -240.51
No. Observations:                  22   AIC:                             485.0
Df Residuals:                      20   BIC:                             487.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          4.872e+04   8350.443      5.835